In [ ]:
#Assignment 3 - Part B: Practical Implementation & Deployment
#By Nanfuka Fatuma 
#B36106.

file 4 PROJECT: Optimizing Food Distribution Scheduling to Refugee Settlements in West Nile, Uganda.

In [ ]:
!pip install fastapi uvicorn pydantic stable-baselines3

In [ ]:
# making the inference API


from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import numpy as np
import os
from stable_baselines3 import PPO

MODEL_PATH = os.path.join(os.getcwd(), '..', '..', 'models', 'ppo_final_model.zip')

app = FastAPI(title="Food Distribution RL Inference API")

class Observation(BaseModel):
    queue_lengths: list
    staff_levels: list
    trucks: int
    time_of_day: int
    weather: int
    supply_levels: list

# load model at startup
@app.on_event("startup")
def load_model():
    global model
    if os.path.exists(MODEL_PATH):
        model = PPO.load(MODEL_PATH)
    else:
        model = None

@app.post('/predict')
def predict(obs: Observation):
    # convert to observation vector
    vec = np.concatenate([
        np.array(obs.queue_lengths, dtype=np.float32),
        np.array(obs.staff_levels, dtype=np.float32),
        np.array([obs.trucks, obs.time_of_day, obs.weather], dtype=np.float32),
        np.array(obs.supply_levels, dtype=np.float32)
    ])
    if model is None:
        return {"error": "Model not found on server. Train and place model at models/ppo_final_model.zip"}

    action, _states = model.predict(vec, deterministic=True)
    return {"action": int(action)}

if __name__ == '__main__':
    uvicorn.run(app, host='0.0.0.0', port=8000)

ModuleNotFoundError: No module named 'fastapi'